In [14]:
# Imports + Configuration
import os
import time
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta

# =========================
# ClickUp Configuration
# =========================
SPACE_ID = "90159483029"
TEAM_ID = "90152198197"  # For template lookups
# TARGET_FOLDER_NAME = "50 HTL | Oitilo Mani"
TARGET_FOLDER_NAME = "DATA CENTERS"
TARGET_LIST_NAME = "MELIGALAS"
# TASK_TEMPLATE_NAME = "HTL | TEMPLATE | Execution"
# TASK_TEMPLATE_ID = "t-86c7t51ff"  # Fallback if name lookup fails
# CSV_PATH_SIMPLIFIED = "OITILO_ClickUP_WBS_simplified.csv"
CSV_PATH_SIMPLIFIED = "dc_meligalas_wbs_dependencies.csv"

CLICKUP_TOKEN = "pk_56660333_WATA0RDNID48ZA30VX30Z2CRSNB16SN3"
DRY_RUN = False
BASE = "https://api.clickup.com/api/v2"

# Authentication
sess = requests.Session()
sess.headers.update({
    "Authorization": CLICKUP_TOKEN,
    "Accept": "application/json",
    "Content-Type": "application/json",
})

In [15]:
# Helper functions (HTTP + time + deps parsing)
def req(method: str, url: str, *, params=None, json=None, ok=(200, 201), retries=6):
    if DRY_RUN:
        print(f"[DRY_RUN] {method} {url} params={params} json={json}")
        return {}
    last = None
    for i in range(retries):
        r = sess.request(method, url, params=params, json=json, timeout=60)
        if r.status_code in ok:
            return r.json() if r.text else {}
        if r.status_code == 429 or r.status_code >= 500:
            wait = min(60, 2 ** i)
            time.sleep(wait)
            last = (r.status_code, r.text[:400])
            continue
        snippet = (r.text or "")[:400]
        raise RuntimeError(f"{method} {url} failed: {r.status_code} {snippet}")
    raise RuntimeError(f"{method} {url} failed after retries. Last error: {last}")

def date_to_ms_midday_utc(date_str: str) -> int:
    d = datetime.strptime(date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc) + timedelta(hours=12)
    return int(d.timestamp() * 1000)

def days_to_ms(days: float) -> int:
    return int(float(days) * 24 * 60 * 60 * 1000)

def split_deps(dep_cell) -> list[str]:
    if dep_cell is None or (isinstance(dep_cell, float) and pd.isna(dep_cell)):
        return []
    s = str(dep_cell).strip()
    if not s or s.lower() == "nan":
        return []
    parts = []
    for chunk in s.replace("；", ";").replace(",", ";").split(";"):
        c = chunk.strip()
        if c:
            parts.append(c)
    return parts

def plot_all_templates(team_id: str = None) -> pd.DataFrame:
    """
    Fetch and display all ClickUp task templates in a formatted table.
    
    Args:
        team_id: ClickUp Team ID. If None, uses TEAM_ID from config.
    
    Returns:
        DataFrame with template info (id, name).
    """
    team_id = team_id or TEAM_ID
    try:
        response = req("GET", f"{BASE}/team/{team_id}/taskTemplate")
        templates = response.get("templates", [])
        
        if not templates:
            print("No templates found.")
            return pd.DataFrame()
        
        data = []
        for t in templates:
            data.append({
                "Template ID": t.get("id", ""),
                "Template Name": t.get("name", ""),
                "Created Date": t.get("date_created", ""),
                "Color": t.get("color", ""),
            })
        
        df_templates = pd.DataFrame(data)
        print(f"\n{'='*80}")
        print(f"Found {len(df_templates)} ClickUp Templates in Team {team_id}")
        print(f"{'='*80}\n")
        print(df_templates.to_string(index=False))
        print(f"\n{'='*80}\n")
        
        return df_templates
    except Exception as e:
        print(f"Error fetching templates: {e}")
        return pd.DataFrame()

In [16]:
# Load CSV
df_s = pd.read_csv(CSV_PATH_SIMPLIFIED)

# Map actual CSV columns to expected names (handle both naming conventions)
column_mapping = {
    "Depends On": "Dependencies",
    "Time Estimated": "Estimated Time (days)",
}

# Apply mapping if columns exist
for old_col, new_col in column_mapping.items():
    if old_col in df_s.columns:
        df_s[new_col] = df_s[old_col]

# Define required columns - some may be optional
required = ["Task Name", "Start Date", "Due Date"]
optional = ["WBS", "Branch", "Dependencies", "Estimated Time (days)", "Description"]

# Check required columns
missing_required = [c for c in required if c not in df_s.columns]
if missing_required:
    raise ValueError(f"CSV missing REQUIRED columns: {missing_required}. Found: {list(df_s.columns)}")

# Fill missing optional columns with defaults
for col in optional:
    if col not in df_s.columns:
        if col in ["WBS", "Branch", "Description"]:
            df_s[col] = ""
        elif col == "Dependencies":
            df_s[col] = None
        elif col == "Estimated Time (days)":
            df_s[col] = None

print(f"CSV columns after mapping: {list(df_s.columns)}")

# Clean data
df_s = df_s.copy()
if "WBS" in df_s.columns:
    df_s["WBS"] = df_s["WBS"].astype(str).str.strip()
if "Branch" in df_s.columns:
    df_s["Branch"] = df_s["Branch"].astype(str).str.strip()
if "Task Name" in df_s.columns:
    df_s["Task Name"] = df_s["Task Name"].astype(str).str.strip()
if "Dependencies" in df_s.columns:
    df_s["Dependencies"] = df_s["Dependencies"].astype(str)

CSV columns after mapping: ['Task ID', 'Task Name', 'Due Date', 'Start Date', 'Time Estimated', 'Time Estimated Text', 'Depends On', 'Dependencies', 'Estimated Time (days)', 'WBS', 'Branch', 'Description']


In [17]:
templates_df = plot_all_templates()


Found 12 ClickUp Templates in Team 90152198197

Template ID                     Template Name Created Date Color
t-86c7gd00b   TEMPLATE | Sheduled Task Parent                   
t-86c7gd05q   TEMPLATE | Scheduled Task Child                   
t-86c7gd0n2      TEMPLATE | Monitoring Parent                   
t-86c7gd0ta       TEMPLATE | Reporting Parent                   
t-86c7gd0w3                  TEMPLATE | Claim                   
t-86c7gd0y3                TEMPLATE | General                   
t-86c7t51ff        HTL | TEMPLATE | Execution                   
t-86c7gd0pb       TEMPLATE | Monitoring Child                   
t-86c7gngg7        TEMPLATE | HQ General Task                   
t-86c7n8hr7                DEV | Typical Task                   
t-86c7mz2n1 Project Intake (Dev Portfolio GR)                   
t-86c7gcxtr             TEMPLATE | Corrective                   




In [18]:
# Sort and prepare data
def safe_dt_s(s):
    try:
        return datetime.strptime(s, "%Y-%m-%d")
    except:
        return datetime(2100, 1, 1)

df_s = df_s.sort_values(
    by=["Start Date", "Task Name"], 
    key=lambda col: col.map(safe_dt_s) if col.name == "Start Date" else col
).reset_index(drop=True)

print(f"Loaded {len(df_s)} tasks from {CSV_PATH_SIMPLIFIED}")

# ===== CREATE TASKS AND LOAD TO CLICKUP =====
name_to_id_s = {}
created_cnt_s = 0
updated_cnt_s = 0

for idx, (_, r) in enumerate(df_s.iterrows(), 1):
    task_name = r["Task Name"]
    wbs_value = r["WBS"] if "WBS" in r else ""
    branch_value = r["Branch"] if "Branch" in r else ""
    
    # Create task - with or without template
    if use_template and template_id:
        created = req(
            "POST",
            f"{BASE}/list/{list_id}/taskTemplate/{template_id}",
            json={"name": task_name},
            ok=(200, 201),
        )
    else:
        # Create task without template
        created = req(
            "POST",
            f"{BASE}/list/{list_id}/task",
            json={"name": task_name},
            ok=(200, 201),
        )
    
    task_id = str(created.get("id") or created.get("task", {}).get("id") or "")
    if not task_id:
        raise RuntimeError(f"Could not parse created task id for '{task_name}'. Response: {created}")
    
    # Store by task name for dependency lookup
    name_to_id_s[task_name] = task_id
    created_cnt_s += 1

    # Build payload for field updates
    payload = {}
    
    # Start Date
    sd = r["Start Date"]
    if sd and str(sd).lower() != "nan":
        payload["start_date"] = date_to_ms_midday_utc(str(sd))
        payload["start_date_time"] = False
    
    # Due Date
    dd = r["Due Date"]
    if dd and str(dd).lower() != "nan":
        payload["due_date"] = date_to_ms_midday_utc(str(dd))
        payload["due_date_time"] = False
    
    # Estimated Time (days)
    est = r.get("Estimated Time (days)")
    if est is not None and not (isinstance(est, float) and pd.isna(est)):
        try:
            payload["time_estimate"] = days_to_ms(float(est))
        except:
            pass
    
    # Update task with all fields
    if payload:
        req("PUT", f"{BASE}/task/{task_id}", json=payload, ok=(200,))
        updated_cnt_s += 1
    
    # Set WBS custom field if field exists and value is valid
    if wbs_field_id and wbs_value and str(wbs_value).lower() not in ["nan", ""]:
        try:
            req(
                "POST",
                f"{BASE}/task/{task_id}/field/{wbs_field_id}",
                json={"value": str(wbs_value)},
                ok=(200,)
            )
        except Exception as e:
            print(f"  ⚠ Could not set WBS for task '{task_name}': {e}")
    
    # Set Branch custom field if field exists and value is valid
    if branch_field_id and branch_value and str(branch_value).lower() not in ["nan", ""]:
        opt_id = branch_options.get(str(branch_value).strip().lower())
        if opt_id:
            try:
                req(
                    "POST",
                    f"{BASE}/task/{task_id}/field/{branch_field_id}",
                    json={"value": opt_id},
                    ok=(200,)
                )
            except Exception as e:
                print(f"  ⚠ Could not set Branch for task '{task_name}': {e}")
        else:
            if branch_value:
                print(f"  ⚠ Branch value '{branch_value}' not in dropdown options: {list(branch_options.keys())}")
    
    print(f"  [{idx}/{len(df_s)}] ✓ Created: {task_name} (id={task_id})")

print(f"\n{'='*80}")
print(f"Created: {created_cnt_s} tasks")
print(f"Updated: {updated_cnt_s} tasks with field values")
print(f"{'='*80}")

# Link dependencies
deps_added_s = 0
deps_missing_s = 0
deps_already_s = 0

for _, r in df_s.iterrows():
    child_name = r["Task Name"]
    child_id = name_to_id_s.get(child_name)
    if not child_id:
        continue
    
    for parent_name in split_deps(r.get("Dependencies")):
        parent_id = name_to_id_s.get(parent_name)
        if not parent_id:
            deps_missing_s += 1
            continue
        try:
            req(
                "POST",
                f"{BASE}/task/{child_id}/dependency",
                json={"depends_on": parent_id},
                ok=(200, 201),
            )
            deps_added_s += 1
        except RuntimeError as e:
            if "already" in str(e).lower() or "exist" in str(e).lower():
                deps_already_s += 1
            else:
                raise

print(f"\n{'='*80}")
print(f"Dependencies linked: {deps_added_s}")
if deps_missing_s > 0:
    print(f"  (Missing parent tasks: {deps_missing_s})")
if deps_already_s > 0:
    print(f"  (Already existed: {deps_already_s})")
print(f"{'='*80}")

print("\n✓✓✓ IMPORT TO CLICKUP COMPLETE! ✓✓✓")

Loaded 60 tasks from dc_meligalas_wbs_dependencies.csv


RuntimeError: POST https://api.clickup.com/api/v2/list/901520527670/task failed: 404 {"err":"List deleted","ECODE":"ACCESS_100","meta":{"authorization_failures":[{"object_id":"901520527670","object_type":"list","workspace_id":90152198197,"code":"DELETED","invalid_permissions":["can_create_lists_pl"]}]}}